In [7]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict, Counter
import math


# Lectura de documentos

In [ ]:
archivo = r"C:\Users\afpue\Documents\GitHub\NLP-Seminar\publicaciones\binaria.xlsx"
df = pd.read_excel(archivo)

def limpiar(texto):
    texto = texto.lower()
    return re.findall(r'\b\w+\b', texto)

docs = df['Documento'].apply(limpiar).tolist()
clases = df['Clase'].tolist()


In [9]:
nuevo_procesado = limpiar(df.iloc[-1]['Documento'])
clase_real = df.iloc[-1]['Clase']

docs = df.iloc[:-1]['Documento'].apply(limpiar).tolist()
clases = df.iloc[:-1]['Clase'].tolist()

# Bnaria naive Bayes

In [ ]:
def train_binary_naive_bayes(D, C):
    """
    Entrena un clasificador Binary Multinomial Naive Bayes con add-1 smoothing.
    En este modelo solo se considera si una palabra aparece al menos una vez por documento.

    Parámetros:
    - D: lista de documentos (listas de palabras)
    - C: lista de clases (una por documento)

    Retorna:
    - V: vocabulario (lista de palabras únicas)
    - logprior: diccionario con log P(c) para cada clase c
    - loglikelihood: diccionario con log P(w|c) para cada palabra w y clase c
    """

    N_doc = len(D)  # Número total de documentos
    logprior = {}   # Diccionario que almacenará log P(c)
    loglikelihood = {}  # Diccionario que almacenará log P(w|c)
    bigdoc = defaultdict(list)  # Diccionario que concatena palabras por clase
    vocab = set()   # Conjunto para construir el vocabulario (sin repeticiones)

    # === Binarizar los documentos: solo cuenta si una palabra aparece (no cuántas veces)
    bin_docs = [list(set(doc)) for doc in D]  # Se eliminan repeticiones dentro de cada documento

    # === Construcción del bigdoc y vocabulario
    for doc, c in zip(bin_docs, C):
        bigdoc[c].extend(doc)     # Agrega las palabras únicas del documento a su clase
        vocab.update(doc)         # Añade palabras al vocabulario general

    V = list(vocab)  # Convertimos el vocabulario a lista (para recorrido ordenado)

    # === Cálculo de logprior y loglikelihood
    for c in bigdoc:
        Nc = C.count(c)  # Número de documentos con clase c
        logprior[c] = math.log(Nc / N_doc)  # log P(c) = log(#docs clase c / #docs totales)

        word_counts = Counter(bigdoc[c])  # Cuenta en cuántos documentos aparece cada palabra en la clase c
        denom = sum(word_counts[w] + 1 for w in V)  # Denominador total con suavizado Laplace

        for w in V:
            num = word_counts[w] + 1  # Numerador con Laplace: número de documentos clase c donde aparece w + 1
            loglikelihood[(w, c)] = math.log(num / denom)  # log P(w|c)

    return V, logprior, loglikelihood  # Retorna vocabulario, log P(c) y log P(w|c)


In [11]:
def test_naive_bayes(testdoc, logprior, loglikelihood, C, V, verbose=True):
    """
    Clasifica un documento con el modelo entrenado.

    Parámetros:
    - testdoc: lista de palabras del documento a clasificar
    - logprior: diccionario con log P(c) para cada clase
    - loglikelihood: diccionario con log P(w|c) para cada palabra w y clase c
    - C: lista de clases posibles
    - V: vocabulario del modelo entrenado
    - verbose: si True, imprime los scores logarítmicos por clase

    Retorna:
    - clase_predicha: clase con mayor score logarítmico
    """

    sum_scores = {}              # Diccionario para guardar el score total por clase
    V_set = set(V)               # Convertimos el vocabulario a conjunto para acceso rápido

    for c in C:                  # Para cada clase posible
        score = logprior[c]      # Iniciar el score con log P(c)
        for word in testdoc:     # Para cada palabra en el documento de prueba
            if word in V_set:    # Solo consideramos palabras que estén en el vocabulario
                # Sumamos log P(w|c); si no está, usamos 0.0 (probabilidad implícita 1)
                score += loglikelihood.get((word, c), 0.0)
        sum_scores[c] = score    # Guardamos el score total de la clase

    if verbose:                  # Si se desea, se imprime el score logarítmico por clase
        print("Log-probabilidades por clase:")
        for c, score in sum_scores.items():
            print(f"  {c}: {score:.4f}")

    # Devolvemos la clase con el score más alto (máxima probabilidad a posteriori)
    return max(sum_scores, key=sum_scores.get)


In [12]:
V, logprior, loglikelihood = train_binary_naive_bayes(docs, clases)

prediccion = test_naive_bayes(nuevo_procesado, logprior, loglikelihood, list(set(clases)), V)

print("Tweet:", nuevo_procesado)
print("Clase real:", clase_real)
print("Predicción:", prediccion)

Log-probabilidades por clase:
  Positivo: -13.8907
  Negativo: -15.2948
Tweet: ['el', 'mundo', 'me', 'parece', 'más', 'amable', 'más', 'humano', 'menos', 'raro']
Clase real: Positivo
Predicción: Positivo


In [13]:
loglikelihood

{('ee', 'Negativo'): -5.159055299214529,
 ('gracias', 'Negativo'): -5.159055299214529,
 ('dólares', 'Negativo'): -5.159055299214529,
 ('miembros', 'Negativo'): -5.159055299214529,
 ('apoyan', 'Negativo'): -5.159055299214529,
 ('princesa', 'Negativo'): -5.159055299214529,
 ('concordia', 'Negativo'): -5.159055299214529,
 ('maneja', 'Negativo'): -4.465908118654584,
 ('para', 'Negativo'): -5.159055299214529,
 ('registra', 'Negativo'): -4.465908118654584,
 ('presencialmente', 'Negativo'): -4.465908118654584,
 ('cuba', 'Negativo'): -5.159055299214529,
 ('suben', 'Negativo'): -4.465908118654584,
 ('caóticos', 'Negativo'): -4.465908118654584,
 ('colombianos', 'Negativo'): -5.159055299214529,
 ('covid', 'Negativo'): -4.060443010546419,
 ('primera', 'Negativo'): -5.159055299214529,
 ('pacientes', 'Negativo'): -5.159055299214529,
 ('7', 'Negativo'): -5.159055299214529,
 ('321', 'Negativo'): -4.465908118654584,
 ('gobierno', 'Negativo'): -5.159055299214529,
 ('intentos', 'Negativo'): -4.4659081186